In [14]:
from sensormap_client import ApiClient, Configuration, models, get_jwt_token
from sensormap_client.api.default_api import DefaultApi
import pandas as pd
import geopandas as gpd
import json

# SERVER = "https://soil-sensormap.epfl.ch" 
SERVER = "http://soil:88"

auth_token = get_jwt_token(SERVER)
config = Configuration(host=SERVER, access_token=auth_token)
api = DefaultApi(ApiClient(configuration=config))

# Test getting some projects, to see if we're connected
print("Connected") if api.get_all_projects() is not None else print("Not connected")

Connected


In [3]:
# Print out the projects and areas
projects = api.get_all_projects()
for project in projects:
    print(f"Project: {project.name}")

areas = api.get_all_areas()
for area in areas:
    print(f"Area: {area.name} (Project: {area.project.name})")

Project: SOC in SAS
Project: Soil Amendment
Area: Blatt (Project: SOC in SAS)
Area: Moudon (Project: Soil Amendment)
Area: Ar du Tsan (Project: SOC in SAS)


In [4]:
# Create a project

# We need to use the ProjectCreate model to create
project = models.ProjectCreate(
    name="Soil Amendment",
    description="The soil amendement project aims at investigatingthe effect of different organic amendements on agricultural soils. This project is in collaboration with the Nestlé research center"
)

# Send it to the Database
created_project = api.create_one_project(project)
print("Created project. ID:", created_project.id)

ConflictException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'Content-Length': '79', 'Content-Type': 'application/json', 'Date': 'Fri, 07 Mar 2025 12:14:13 GMT'})
HTTP response body: "Conflict: duplicate key value violates unique constraint \"project_name_key\""


In [18]:
params = {"name": "Moudon"}
api.get_all_areas(filter=json.dumps(params))

[Area(description=None, geom={'features': [], 'type': 'FeatureCollection'}, id='d8ad9124-b5e0-45e8-96e0-95447bc3aa4f', last_updated=datetime.datetime(2025, 3, 7, 9, 7, 49, 442682, tzinfo=TzInfo(UTC)), name='Moudon', plots=[], project=Project(color='#4052b4', description='The soil amendement project aims at investigatingthe effect of different organic amendements on agricultural soils. This project is in collaboration with the Nestlé research center', id='e75c5630-c114-4ff6-aed8-0e0f4330cb88', last_updated=datetime.datetime(2025, 3, 7, 9, 7, 48, 527735, tzinfo=TzInfo(UTC)), name='Soil Amendment'), project_id='e75c5630-c114-4ff6-aed8-0e0f4330cb88', sensor_profiles=[], soil_profiles=[], transects=[])]

In [8]:
api.get_all_areas

Help on method get_all_areas in module sensormap_client.api.default_api:

get_all_areas(
    filter: Annotated[Optional[Annotated[str, Strict(strict=True)]], FieldInfo(annotation=NoneType, required=True, description='JSON-encoded filter for querying resources.  This parameter supports various filtering options: - Free text search: `{"q": "search text"}` - Filtering by a single ID: `{"id": "550e8400-e29b-41d4-a716-446655440000"}` - Filtering by multiple IDs: `{"id": ["550e8400-e29b-41d4-a716-446655440000", "550e8400-e29b-41d4-a716-446655440001"]}` - Filtering on other columns: `{"name": "example"}`')] = None,
    range: Annotated[Optional[Annotated[str, Strict(strict=True)]], FieldInfo(annotation=NoneType, required=True, description='Range for pagination in the format "[start, end]".  Example: `[0,9]`')] = None,
    sort: Annotated[Optional[Annotated[str, Strict(strict=True)]], FieldInfo(annotation=NoneType, required=True, description='Sort order for the results in the format \'["column

In [19]:
help(models.PlotCreate)

Help on class PlotCreate in module sensormap_client.models.plot_create:

class PlotCreate(pydantic.main.BaseModel)
 |  PlotCreate(
 |      *,
 |      area_id: Annotated[str, Strict(strict=True)],
 |      aspect: Optional[Annotated[str, Strict(strict=True)]] = None,
 |      coord_x: Union[Annotated[float, Strict(strict=True)], Annotated[int, Strict(strict=True)]],
 |      coord_y: Union[Annotated[float, Strict(strict=True)], Annotated[int, Strict(strict=True)]],
 |      coord_z: Union[Annotated[float, Strict(strict=True)], Annotated[int, Strict(strict=True)]],
 |      created_on: Optional[datetime.date] = None,
 |      gradient: sensormap_client.models.gradientchoices.Gradientchoices,
 |      image: Optional[Annotated[str, Strict(strict=True)]] = None,
 |      lithology: Optional[Annotated[str, Strict(strict=True)]] = None,
 |      name: Annotated[str, Strict(strict=True)],
 |      topography: Optional[Annotated[str, Strict(strict=True)]] = None,
 |      vegetation_type: Optional[Annota

In [5]:
area = models.AreaCreate(name="Moudon", project_id=created_project.id)
created_area = api_client.create_one_area(area)

NameError: name 'created_project' is not defined

In [91]:
df = pd.read_csv("moudon.csv")
coords = pd.read_csv("moudon-coords.csv")
coords = gpd.GeoDataFrame(coords, geometry=gpd.GeoSeries.from_xy(coords['longitude'], coords['latitude']), crs=4326).to_crs(2056)

In [92]:
coords[['plot', 'geometry']]

,plot,geometry
0,GV-T6-C,POINT (2548098.166 1168996.112)
1,GV-T6-B,POINT (2548117.51 1169020.649)
2,GV-T6-A,POINT (2548147.643 1169063.622)
3,GV-T5-C,POINT (2548121.521 1168992.822)
4,GV-T5-B,POINT (2548153.726 1169029.6)
5,GV-T5-A,POINT (2548192.387 1169075.587)
6,GV-T4-A,POINT (2548207.319 1169081.634)
7,GV-T4-B,POINT (2548166.506 1169032.578)
8,GV-T4-C,POINT (2548136.427 1168995.781)
9,GV-T3-C,POINT (2548164.006 1168989.366)


In [94]:
df

,No.,Treatment,Plot,PlotID,Replicate,Depth,SOC,BD,N,C,...,Silt,Sand,DNA,bact,arc,fungi,CO2_cum,CO2_flux,arc/bact,fungi/bact
0,1,B-T1,B-T1-A,A,1,0-10,NaN,NaN,0.249278,2.638883,...,50.391155,35.360105,37.426301,1.482526e+10,1.640558e+10,2.621550e+08,NaN,NaN,0.106167,0.017683
1,2,B-T1,B-T1-B,B,2,0-10,NaN,NaN,0.225657,2.342029,...,NaN,NaN,44.330378,1.568562e+10,9.995403e+09,2.938137e+08,NaN,NaN,0.094156,0.018731
2,3,B-T1,B-T1-C,C,3,0-10,NaN,NaN,0.253724,3.265311,...,64.608309,16.761522,48.683789,1.614047e+10,1.236894e+10,2.315638e+08,NaN,NaN,0.108760,0.014347
3,4,B-T2,B-T2-A,A,1,0-10,NaN,NaN,0.129033,1.564225,...,53.936999,36.607290,15.126650,6.151029e+09,1.583629e+10,1.638720e+08,NaN,NaN,0.055576,0.026641
4,5,B-T2,B-T2-B,B,2,0-10,NaN,NaN,0.124588,1.497516,...,NaN,NaN,21.792662,8.689978e+09,6.387471e+09,2.122630e+08,NaN,NaN,0.039445,0.024426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,54,GV-T2,GV-T2-C,C,3,30-50,27.523737,1.767603,0.064414,0.778561,...,NaN,NaN,NaN,9.440767e+08,5.322682e+09,1.556945e+07,NaN,NaN,0.105389,0.016492
62,57,GV-T3,GV-T3-C,C,3,30-50,35.398789,1.718436,0.086023,1.029971,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,60,GV-T4,GV-T4-C,C,3,30-50,16.507217,1.749628,0.034187,0.471735,...,NaN,NaN,NaN,1.784138e+09,7.657702e+09,4.062822e+07,NaN,NaN,0.088557,0.022772
64,63,GV-T5,GV-T5-C,C,3,30-50,22.282757,1.652601,0.050542,0.674173,...,NaN,NaN,NaN,8.563604e+08,9.522292e+09,4.113156e+07,NaN,NaN,0.106451,0.048031


In [100]:
combined = df.merge(coords[['plot', 'geometry']], left_on='Plot', right_on='plot')

In [104]:
api.get_all_areas()

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Length': '66', 'Content-Type': 'application/json', 'Date': 'Fri, 07 Mar 2025 12:13:27 GMT'})
HTTP response body: {"error":"The JWT could not be decoded. Source: InvalidSignature"}


In [103]:
for row in combined.iterrows():
    plot = models.PlotCreate(area_id=are

(0, No.                                                    13
Treatment                                           GV-T1
Plot                                              GV-T1-A
PlotID                                                  A
Replicate                                               1
Depth                                                0-10
SOC                                             28.458201
BD                                               1.174894
N                                                0.685029
 C                                               2.422193
pH                                                   6.89
Mg                                                 8359.0
Mn                                                 1042.0
Fe                                                23680.0
Ni                                                   33.2
Cu                                                   13.1
Zn                                                   58.6
Mo        

In [46]:
unique_plots = df["Treatment"].unique()

In [62]:
for plot in unique_plots:
    print(plot)
    # plot_create = models.PlotCreate(area_id=created_area.id,

BT1
BT2
BT3
BR
GV-T1
GV-T2
GV-T3
GV-T4
GV-T5
GV-T6


In [ ]:
areas = api_client.get_all_areas()

In [ ]:
sensormap_client.mo

In [ ]:
api_client.create_one_area()